In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [63]:
model_path = "/mnt/geogpt-gpfs/llm-course/public/models/llama3/Meta-Llama-3-8B-Instruct/"

model = AutoModelForCausalLM.from_pretrained(
    model_path, 
    device_map="cuda:0",
    torch_dtype=torch.bfloat16,
    load_in_4bit=True,
    # attn_implementation="flash_attention_2",
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [64]:
tokenizer = AutoTokenizer.from_pretrained(model_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [65]:
len(tokenizer)

128256

In [66]:
tokenizer.pad_token_id = 128001

In [67]:
# Format message with the command-r chat template
messages = [
    [{"role": "user", "content": "As someone who's been following the advancements of observational cosmology closely, the issue of host galaxy mismatches for Type Ia Supernovae when deducing cosmological parameters has been quite pertinent. Considering the efforts made by the researchers in using the Dark Energy Survey 5-Year photometric sample, I would like to understand in detail how host galaxy mismatches can influence the results? Furthermore, the methodology utilized in this study intrigues me. Could you elaborate on the process of creating the deep galaxy catalog and the significance of photometric redshifts as well as the role of spectroscopic information in ensuring the fidelity of the catalog matching? Also, how were the simulations structured to model these host galaxy mismatches, and what findings about the impact on cosmological parameters were unearthed? It would be a bonus to comprehend the measures taken to account for host confusion and its contribution to the biases in question. Lastly, the steps taken after identifying mismatches for improving the accuracy of future measurements warrant discussion too. This investigation seems to carry substantial weight for the astrophysics community and your detailed explanation would be much appreciated."}]
]
input_ids = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")
## <BOS_TOKEN><|START_OF_TURN_TOKEN|><|USER_TOKEN|>Hello, how are you?<|END_OF_TURN_TOKEN|><|START_OF_TURN_TOKEN|><|CHATBOT_TOKEN|>
input_ids = input_ids.to("cuda:0")
attention_mask = input_ids.ne(tokenizer.pad_token_id).long()


gen_tokens = model.generate(
    input_ids, 
    attention_mask=attention_mask,
    max_new_tokens=1024, 
    do_sample=True, 
    temperature=0.3,
    )

# gen_text = tokenizer.decode(gen_tokens[0])
# print(gen_text)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [68]:
gen_text = tokenizer.decode(gen_tokens[0])
gen_text

'<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nAs someone who\'s been following the advancements of observational cosmology closely, the issue of host galaxy mismatches for Type Ia Supernovae when deducing cosmological parameters has been quite pertinent. Considering the efforts made by the researchers in using the Dark Energy Survey 5-Year photometric sample, I would like to understand in detail how host galaxy mismatches can influence the results? Furthermore, the methodology utilized in this study intrigues me. Could you elaborate on the process of creating the deep galaxy catalog and the significance of photometric redshifts as well as the role of spectroscopic information in ensuring the fidelity of the catalog matching? Also, how were the simulations structured to model these host galaxy mismatches, and what findings about the impact on cosmological parameters were unearthed? It would be a bonus to comprehend the measures taken to account for host confusion and its

In [6]:
for i in range(len(messages)):
    gen_text = tokenizer.decode(gen_tokens[i])
    print(messages[i][0]['content'])
    print(gen_text.split('<|CHATBOT_TOKEN|>')[1].split("<|END_OF_TURN_TOKEN|>")[0])
    print('\n\n\n\n')

As someone who's been following the advancements of observational cosmology closely, the issue of host galaxy mismatches for Type Ia Supernovae when deducing cosmological parameters has been quite pertinent. Considering the efforts made by the researchers in using the Dark Energy Survey 5-Year photometric sample, I would like to understand in detail how host galaxy mismatches can influence the results? Furthermore, the methodology utilized in this study intrigues me. Could you elaborate on the process of creating the deep galaxy catalog and the significance of photometric redshifts as well as the role of spectroscopic information in ensuring the fidelity of the catalog matching? Also, how were the simulations structured to model these host galaxy mismatches, and what findings about the impact on cosmological parameters were unearthed? It would be a bonus to comprehend the measures taken to account for host confusion and its contribution to the biases in question. Lastly, the steps take

In [30]:
with open("./datasets/question_astro.txt", 'r') as f:
    lines = f.read().splitlines()
lines = [line.split('.')[1].strip() for line in lines]
lines[:10]

['What is the difference between a comet and an asteroid?',
 'How does the Hubble Space Telescope differ from ground-based telescopes?',
 'Explain the lifecycle of a star like our Sun',
 'What are the main components of the Milky Way galaxy?',
 'How do black holes form?',
 'What is the significance of the cosmic microwave background radiation?',
 'How do astronomers measure the distance to distant stars?',
 'What is dark matter and why is it important in cosmology?',
 'Explain the concept of redshift and how it relates to the expanding universe',
 'What is the Hertzsprung-Russell diagram and how is it used in astronomy?']

In [31]:
with open("./datasets/question_astro.txt", 'w') as f:
    f.write('\n'.join(lines))

In [33]:
gen_text = tokenizer.decode(gen_tokens[0], skip_special_tokens=True)

In [34]:
gen_text

'<|START_OF_TURN_TOKEN|><|USER_TOKEN|>What is the principle of period modulation of RRab variable stars<|START_OF_TURN_TOKEN|><|CHATBOT_TOKEN|>RR Lyrae variable stars, specifically those classified as RRab type, exhibit a phenomenon known as period modulation. This intriguing behavior arises from complexities in the stars\' pulsation mechanisms and likely reflects variations in the stellar environment. The fundamental principle behind period modulation can be summarized as follows:\n\nPeriod modulation in RRab variables refers to observable changes in the stars\' pulsation periods over time. These variations in period are believed to originate from cyclical alterations in the stars\' radial fundamental mode pulsations. RRab stars typically pulsate with a characteristic period, but this period is not constant and can exhibit short-term and long-term fluctuations.\n\nThe primary theory behind period modulation revolves around the concept of a "period doubling window." This window corresp

In [16]:
from accelerate import Accelerator

accelerator = Accelerator()
accelerator.free_memory()

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


[]